In [0]:
%pip install openai langchain databricks-langchain langchain_core langchain-community mlflow
dbutils.library.restartPython()

In [0]:
import os
from openai import OpenAI 

In [0]:
dbutils.widgets.text("workspace_url", "")
dbutils.widgets.text("base_llm_model", "databricks-meta-llama-3-3-70b-instruct")
base_url = dbutils.widgets.get("workspace_url")
base_llm_model = dbutils.widgets.get("base_llm_model")
embedding_model = "databricks-gte-large-en"


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [0]:
#Env Setup for OpenAI token
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url=base_url
)

In [0]:
def get_completion(prompt: str, model: str = base_llm_model):
  completions = client.chat.completions.create(
    model = model,
    #store = True,
    temperature = 0,
    messages = [
      {"role": "user", "content": prompt}
    ],
    
  )
  return completions.choices[0].message.content

In [0]:
get_completion("What is ML?")

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [0]:
style = """American English \
in a calm and respectful tone
"""

In [0]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [0]:
response = get_completion(prompt)
print(response)

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [0]:
from databricks_langchain import ChatDatabricks

In [0]:
chat = ChatDatabricks(
    endpoint = base_llm_model,
    extra_params = {"temperature": 0.1}
)
chat

### Prompt template

In [0]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [0]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [0]:
prompt_template.messages[0].prompt

In [0]:
prompt_template.messages[0].prompt.input_variables

In [0]:
customer_style = """American English \
in a calm and respectful tone
"""

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [0]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

In [0]:
type(customer_messages[0])

In [0]:
print(customer_messages[0])

In [0]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [0]:
print(customer_response.content)

* One more Example

In [0]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [0]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [0]:
service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)

print(service_messages[0].content)

In [0]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers

Let's start with defining how we would like the LLM output to look like: